In [8]:
%%writefile function_code/requirements.txt
google-cloud-videointelligence
google-cloud-video-transcoder
google-cloud-storage

Writing function_code/requirements.txt


In [31]:
%%writefile function_code/main.py

import json

from google.cloud import storage
from google.cloud.video import transcoder

from Annotations_util import get_segments_for_labels
from Transcoder_utils import create_fade_overlay_segment, create_video_overlay_job, create_static_overlay_segment


OUTPUT_BUCKET = 'gs://scary-videos-fixed/'
parent_location ='projects/320823659895/locations/europe-west1'

transcoder_client = transcoder.TranscoderServiceClient()
storage_client = storage.Client()


def create_safe_video(event, context):
    
    file_name = event['name']
    bucket_name = event['bucket']
    print(file_name)
    
    # split the fears off the end of the file name name after 🙈 and separate on '&'    
    fears = set(file_name.split('.')[-2].split('🙈')[-1].split('+'))
    print('look for fears', fears)
    fears = set({'bird'})
    
    annotaion_bucket = storage_client.get_bucket(bucket_name)
    
    # downalod annotations from GCS     
    source_blob_name = file_name
    blob = annotaion_bucket.blob(source_blob_name)
    text = blob.download_as_string()
    annotation_results = json.loads(text)
    
    # get original file name     
    input_uri = 'gs:/' + annotation_results['annotation_results'][0]['input_uri']
    file_name = input_uri.split('/')[-1].split('.')[0]
    
    
    # print(annotation_results)
    segments = get_segments_for_labels(fears, annotation_results['annotation_results'][0]['shot_label_annotations'])
    
    print(segments)
    
    
    # create all the overlay segments     
    overlay_segments = []
    for seg in segments:
        overlay_segments += create_static_overlay_segment(*seg)
    
    print(overlay_segments)
    
    job = create_video_overlay_job(input_url=input_uri, 
                                   output_folder=OUTPUT_BUCKET, 
                                   output_file_name=file_name, 
                                   overlay_segments = overlay_segments)

    job_request = transcoder.CreateJobRequest(parent = parent_location, job = job)
    
    transcoder_client.create_job(job_request)
    

Writing function_code/main.py


In [29]:
event ={'bucket': 'scary-video-annotations', 'name': 'zoo🙈giraffe+rhinoceros.json'}
create_safe_video(event, None)

zoo🙈giraffe+rhinoceros.json
look for fears {'giraffe', 'rhinoceros'}
[animation_static {
  xy {
  }
  start_time_offset {
    seconds: 7
    nanos: 480000000
  }
}
, animation_end {
  start_time_offset {
    seconds: 8
    nanos: 880000000
  }
}
, animation_static {
  xy {
  }
  start_time_offset {
    seconds: 8
    nanos: 919999999
  }
}
, animation_end {
  start_time_offset {
    seconds: 9
    nanos: 720000000
  }
}
, animation_static {
  xy {
  }
  start_time_offset {
    seconds: 29
    nanos: 480000000
  }
}
, animation_end {
  start_time_offset {
    seconds: 32
    nanos: 560000000
  }
}
, animation_static {
  xy {
  }
  start_time_offset {
    seconds: 53
    nanos: 439999999
  }
}
, animation_end {
  start_time_offset {
    seconds: 55
    nanos: 200000000
  }
}
, animation_static {
  xy {
  }
  start_time_offset {
    seconds: 115
    nanos: 840000000
  }
}
, animation_end {
  start_time_offset {
    seconds: 118
  }
}
]


In [30]:
labels

In [32]:
!gcloud functions deploy create_safe_video \
--region europe-west2 \
--runtime python37 \
--source function_code \
--entry-point create_safe_video \
--timeout 60 \
--retry \
--trigger-resource gs://scary-video-annotations \
--trigger-event google.storage.object.finalize

Deploying function (may take a while - up to 2 minutes)...⠼                    
For Cloud Build Stackdriver Logs, visit: https://console.cloud.google.com/logs/viewer?project=phobia-safe-video-player&advancedFilter=resource.type%3Dbuild%0Aresource.labels.build_id%3D8abb997e-bacb-4642-b3a9-d6bb44f1b4fb%0AlogName%3Dprojects%2Fphobia-safe-video-player%2Flogs%2Fcloudbuild
Deploying function (may take a while - up to 2 minutes)...done.                
availableMemoryMb: 256
buildId: 8abb997e-bacb-4642-b3a9-d6bb44f1b4fb
entryPoint: create_safe_video
eventTrigger:
  eventType: google.storage.object.finalize
  failurePolicy:
    retry: {}
  resource: projects/_/buckets/scary-video-annotations
  service: storage.googleapis.com
ingressSettings: ALLOW_ALL
labels:
  deployment-tool: cli-gcloud
name: projects/phobia-safe-video-player/locations/europe-west2/functions/create_safe_video
runtime: python37
serviceAccountEmail: phobia-safe-video-player@appspot.gserviceaccount.com
sourceUploadUrl: https://